In [1]:
import pandas as pd
import numpy as np

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

/opt/anaconda3/lib/python3.9/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/opt/anaconda3/lib/python3.9/site-packages/seaborn/rcmod.py:400: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [2]:
import nltk
stop_words = nltk.corpus.stopwords.words("english")
for word in ['what', 'how', 'where', 'who', 'which'] :
    stop_words.append(word)
from string import punctuation

In [11]:
from bs4 import BeautifulSoup

In [3]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [11]:
import spacy

In [12]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [4]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
file = open("top_10_tags.txt", "r")
top_10_tags = file.read()
top_10_tags = list(top_10_tags.split('\n')[:-1])
file.close()

In [7]:
data = pd.read_csv("data.csv")
data = data.drop(columns = data.columns[0])

In [8]:
text = data['Body']
text_spl = text.sample(frac = 0.25).reset_index(drop = True)
text_spl.head()

0    <p>Does anyone know the specifics of how to em...
1    <p>I currently have a script that reads the ex...
2    <p>I have my React client post a file with the...
3    <p>From googling around it looks like Xcode (3...
4    <p>Is there a way to animate a change in the f...
Name: Body, dtype: object

In [17]:
print("Textes bruts :")
print("")
print(text_spl[:11])
print("---------------------------------------")
print("Textes nettoyés par Gensim :")
print("")
print(text_spl[:11].apply(simple_preprocess))

Textes bruts :

0     <p>I'm trying to build a Python daemon that la...
1     <p>I am in this very unfortunate situation:</p...
2     <blockquote>\n  <p>The problem</p>\n</blockquo...
3     <p>I'm working with Vaadin framework, and it h...
4     <p><strong>Short version</strong> for the hast...
5     <p>I have a Core Data entity, "bid", which has...
6     <p>I have data points whose abscissas are <cod...
7     <p>I'm trying to use the NSSavePanel and added...
8     <p>I have 3 types of data to validate</p>\n\n<...
9     <p>My problem is: I need to overload standard ...
10    <p>When running my project in XCode, my tests ...
Name: Body, dtype: object
---------------------------------------
Textes nettoyés par Gensim :

0     [trying, to, build, python, daemon, that, laun...
1     [am, in, this, very, unfortunate, situation, u...
2     [blockquote, the, problem, blockquote, my, doc...
3     [working, with, vaadin, framework, and, it, ha...
4     [strong, short, version, strong, for, the,

In [18]:
def lemmatization(texts, allowed_postags = ["NOUN", "VERB", "ADJ", "ADV"]) :
    nlp = spacy.load("en_core_web_sm", disable = ["parser", "ner"])
    texts_out = []
    for text in texts :
        doc = nlp(text)
        new_text = []
        for token in doc :
            if token.orth_ in top_30_tags :
                new_text.append(token.orth_)
            else :
                if token.pos_ in allowed_postags :
                    new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return texts_out

In [19]:
print("Textes bruts :")
print("")
print(text_spl[:11])
print("---------------------------------------")
print("Textes nettoyés par spaCy :")
print("")
print(pd.Series(lemmatization(text_spl[:11])))

Textes bruts :

0     <p>I'm trying to build a Python daemon that la...
1     <p>I am in this very unfortunate situation:</p...
2     <blockquote>\n  <p>The problem</p>\n</blockquo...
3     <p>I'm working with Vaadin framework, and it h...
4     <p><strong>Short version</strong> for the hast...
5     <p>I have a Core Data entity, "bid", which has...
6     <p>I have data points whose abscissas are <cod...
7     <p>I'm trying to use the NSSavePanel and added...
8     <p>I have 3 types of data to validate</p>\n\n<...
9     <p>My problem is: I need to overload standard ...
10    <p>When running my project in XCode, my tests ...
Name: Body, dtype: object
---------------------------------------
Textes nettoyés par spaCy :

0     i'm try build daemon launch other fully indepe...
1     very unfortunate website use outdated software...
2     p docker compose stack comprise postgresql red...
3     i'm work framework poor support intercept even...
4     various table entity domain model same fiel

In [14]:
def preprocess(text) :

    """" Nettoyage du texte :
    passage au minuscule
    suppression du code éventuel du texte que l'on stocke dans une variable 'code'
    suppression et du contenu des balises autres que p (script, alt, ...)
    suppression des balises html
    conservation des textes labellisés par les top 30 tags uniquement
    suppression de la ponctuation, des chiffres,
    et des stopwords
    lemmatisation par spaCy """
    
    text = text.lower()
    
    soup = BeautifulSoup(text)
    
    if soup.find("code") :        
        code = soup.find("code").get_text()
        soup.find('code').clear()
    text_wo_tags = soup.get_text()
    
    for i in range(1, len(text_wo_tags)) :
        if text_wo_tags[i-1] == 'c' and text_wo_tags[i] == '#' :
            text_wo_tags = text_wo_tags.replace(text_wo_tags[i], 'sharp')
    
    token_list = nltk.word_tokenize(text_wo_tags)
    
    new_text = []
    
    for token in token_list :
        if token in top_10_tags :
            new_text.append(token)
        elif token not in stop_words and len(token) > 2 :
            for char in token :
                if char in punctuation or char.isdigit() :
                    token = token.replace(char, '')
            new_text.append(token)
    
    lem = nltk.stem.WordNetLemmatizer()
    
    for token in new_text :
        if nltk.pos_tag([token])[0][1].startswith('V') :
            index = new_text.index(token)
            token_lem = lem.lemmatize(token, pos = 'v')
            new_text[index] = new_text[index].replace(token, token_lem)
            
    new_text = ' '.join(new_text)

    return new_text

In [15]:
print("Textes bruts :")
print("")
print(text_spl[:11])
print("---------------------------------------")
print("Textes nettoyés par la fonction créée :")
print("")
print(text_spl[:11].apply(preprocess))

Textes bruts :

0     <p>Does anyone know the specifics of how to em...
1     <p>I currently have a script that reads the ex...
2     <p>I have my React client post a file with the...
3     <p>From googling around it looks like Xcode (3...
4     <p>Is there a way to animate a change in the f...
5     <p>I have to add <code>UIWebView</code> and so...
6     <p>I've been searching for some good tutorial ...
7     <p>I'm following this <a href="http://download...
8     <p>I am looking for a definitive specification...
9     <p>My Razor views in VS2015 RC are not showing...
10    <p>I found this great snippet from this <a hre...
Name: Body, dtype: object
---------------------------------------
Textes nettoyés par la fonction créée :

0     anyone know specifics embed mono runtime delph...
1     currently script reads exist version csv save ...
2     react client post file fetch api dataset endpo...
3     google around looks like xcode  case least try...
4     way animate change font size te

In [16]:
%%time
text_clean = text_spl.parallel_apply(preprocess)

CPU times: user 970 ms, sys: 200 ms, total: 1.17 s
Wall time: 21.5 s


In [17]:
words = []
for doc in text_clean :
    words.append(nltk.word_tokenize(doc))

In [18]:
id2word = corpora.Dictionary(words)
corpus = []

for word in words :
    corpus.append(id2word.doc2bow(word))

In [35]:
def coherence_table(corpus, dictionary, list_n):
    
    coherence_table = []
    
    for i in list_n :
    
        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=i, 
                                               random_state=100,
                                               chunksize=100,
                                               passes=10)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=id2word, coherence='c_v')

        coherence_table.append(coherence_model_lda.get_coherence())

    return coherence_table

In [ ]:
%%time
n_topics_range = np.linspace(3, 30, 10)
table = coherence_table(corpus, id2word, n_topics_range)

In [ ]:
sns.lineplot(y = table, x = n_topics_range).set(xlabel = "n_topics", ylabel = "Cohérence")
plt.title("Score de cohérence du modèle de LDA en fonction du nombre de topics")
plt.show()

In [42]:
num_topics = 20

lda_multi = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100)

In [43]:
pyLDAvis.enable_notebook()
gensimvis.prepare(lda_multi, corpus, id2word, mds = 'mmds', R=30)

/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: 

/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/opt/anaconda3/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__v

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
18    -0.031356  0.054000       1        1  9.546825
14    -0.004047 -0.044066       2        1  8.324374
13     0.016229  0.059760       3        1  6.877586
8     -0.040093 -0.014368       4        1  6.460301
0     -0.025042  0.020683       5        1  6.407826
3     -0.009620  0.065236       6        1  5.732069
1      0.059974  0.003921       7        1  5.325859
11    -0.062095  0.013536       8        1  5.213883
4      0.021003  0.022461       9        1  5.091397
7     -0.003977 -0.001584      10        1  4.982733
2     -0.073203 -0.018281      11        1  4.915628
19     0.057278 -0.065672      12        1  3.924833
15    -0.059077  0.042428      13        1  3.920038
9      0.030971 -0.015730      14        1  3.751302
5      0.062315 -0.024847      15        1  3.711436
12     0.060439  0.029470      16        1  3.587490
6      0.049964  0.061841      17        1  3.209355
10     0.018185 -0.068698      18        1  3.200825
16    -0.039754 -0.045403      19        1  3.162347
17    -0.028093 -0.074687      20        1  2.653891, topic_info=          Term         Freq        Total Category  logprob  loglift
1029   android  3010.000000  3010.000000  Default  30.0000  30.0000
45    function  3278.000000  3278.000000  Default  29.0000  29.0000
403     public  3878.000000  3878.000000  Default  28.0000  28.0000
1390      list  1684.000000  1684.000000  Default  27.0000  27.0000
874      value  2588.000000  2588.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
71        make    67.111441  2662.808808  Topic20  -5.9765  -0.0516
241     create    67.875172  2883.355160  Topic20  -5.9651  -0.1199
162       want    71.698983  3563.004422  Topic20  -5.9103  -0.2767
434       call    64.119532  2588.651177  Topic20  -6.0221  -0.0690
403     public    65.260267  3878.178361  Topic20  -6.0044  -0.4556

[1700 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
752       1  0.098146  .net
752       2  0.048181  .net
752       3  0.067810  .net
752       4  0.069594  .net
752       5  0.058887  .net
...     ...       ...   ...
2713     11  0.007809   ├──
2713     14  0.070280   ├──
2713     15  0.015618   ├──
2713     19  0.007809   ├──
7681     16  0.806145   👉😀👈

[6467 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 15, 14, 9, 1, 4, 2, 12, 5, 8, 3, 20, 16, 10, 6, 13, 7, 11, 17, 18])